In [1]:
import pandas as pd
import joblib

from datetime import datetime as dtt
from forex_python.converter import CurrencyRates
from collections import Counter

## EDA

In [2]:
df = pd.read_excel('train_data/Hackatondata.xls')

In [44]:
# Select features for training
cols =['declaratie_nr', 'decl_datum_invoer', 'decl_datum_ingediend', 'huidig_declaratie_omschrijving', 
       'log_reden_wijziging', 'dde_omschrijving']
cols_dataset = ['declaratie_nr', 'decl_datum_invoer', 'decl_datum_ingediend', 'decl_valuta_code','decl_verw_bedr_val_vreem',
                'decl_verwacht_bedrag', 'dde_omschrijving', 'huidig_declaratie_status']
data = df[cols_dataset].drop_duplicates('declaratie_nr', keep='last')

data.decl_valuta_code.fillna(inplace=True, value='EUR')

mask = data.decl_verw_bedr_val_vreem.isna()
data.decl_verw_bedr_val_vreem[mask] = data.decl_verwacht_bedrag[mask].copy()

data.columns = ['ID', 'DatumInvoer', 'DatumIngediend', 'Valuta', 'BedragVreemd', 'BedragVerwacht', 'Categorie', 'Status']

data = data.dropna()

# types
data.DatumInvoer = pd.to_datetime(data.DatumInvoer)
data.DatumIngediend = pd.to_datetime(data.DatumIngediend)
data.Status = data.Status.astype(int)

In [45]:
a = list(data.Valuta.value_counts().head().index)

vmap = {'Parkeerkosten': 'Parking', 'Hotelkosten': 'Hotel', 'VOG / uittreksel GBA overig': 'VOG', 'Opleidingskosten': 'Certificatescosts', 'Reiskosten YP': 'Travelcosts'}

In [46]:
for i, r in data.iterrows():
    v = r['Categorie'] 
    v2 = r['Valuta']
    data.set_value(i, 'Categorie', vmap[v] if v in vmap else 'Other')
    data.set_value(i, 'Valuta',  v2 if v2 in a else 'Other')

/home/autarrsd/anaconda3/envs/side/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.
/home/autarrsd/anaconda3/envs/side/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """


In [49]:
def useful(x):
    if x == 90:
        return True
    elif x > 90:
        return False
    else:
        return np.nan

In [50]:
data['Label'] = data.Status.apply(useful)

data = data.dropna(subset=['Label'])

data['DateDiff'] = (data.DatumIngediend - data.DatumInvoer).apply(lambda x: x.days)

data = data.drop(columns=['ID', 'Status', 'DatumInvoer', 'DatumIngediend'])

combined = pd.concat([data[['BedragVreemd', 'BedragVerwacht', 'Label', 'DateDiff']], pd.get_dummies(data.Valuta), 
           pd.get_dummies(data.Categorie)], axis=1)

### Classifier

In [57]:
X.columns

Index(['BedragVreemd', 'BedragVerwacht', 'DateDiff', 'EUR', 'GBP', 'INR',
       'Other', 'SEK', 'USD', 'Certificatescosts', 'Hotel', 'Other', 'Parking',
       'Travelcosts', 'VOG'],
      dtype='object')

In [53]:
from imblearn.over_sampling import RandomOverSampler
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from xgboost import XGBRegressor 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

y = combined.Label.astype(int)
X = combined.drop(columns=['Label'])

ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X, y)
X_train, X_validation, y_train, y_validation = train_test_split(X_resampled, y_resampled, test_size=0.2,
                                                           random_state=42, stratify=y_resampled)

xgb = XGBClassifier(max_depth=25, n_estimators=500, n_jobs=8)
xgb.fit(X_train, y_train)

y_pred = xgb.predict(X_validation)
accuracy_score(y_validation, y_pred)

0.902734375

### Inference

In [110]:
res = pd.DataFrame()
res['Pred'] = y_pred
res['Prob'] = y_pred_proba[:,1]

In [111]:
len(res)

2560

In [82]:
len(res[(res.Prob < 0.2 ) | (res.Prob > 0.8)])

545

In [113]:
def def_action(x, thresh=0.8):
    if x > thresh:
        return 'Accept'
    elif x < 1-thresh:
        return 'Decline'
    else:
        return 'Expert'

In [115]:
res['Action'] = res.Prob.apply(def_action)

In [116]:
Counter(res.Action)

Counter({'Accept': 1067, 'Decline': 1260, 'Expert': 233})

### Save model

In [118]:
import joblib

In [119]:
# Saving model
joblib.dump(xgb, 'xgb_inference.joblib')

['xgb_inference.joblib']

In [120]:
# Loading model
xgb1 = joblib.load('xgb_inference.joblib')

In [124]:
X_validation

array([[128.65, 128.65,   0.  , ...,   0.  ,   0.  ,   0.  ],
       [255.2 , 255.2 ,   0.  , ...,   0.  ,   0.  ,   0.  ],
       [ 72.6 ,  72.6 ,   0.  , ...,   0.  ,   0.  ,   0.  ],
       ...,
       [ 59.  ,  59.  ,   0.  , ...,   0.  ,   0.  ,   0.  ],
       [ 46.13,  46.13,   0.  , ...,   0.  ,   0.  ,   0.  ],
       [ 18.15,  18.15,   0.  , ...,   0.  ,   0.  ,   0.  ]])

In [121]:
# Making sure it still works
y_pred = xgb1.predict(X_validation)
accuracy_score(y_validation, y_pred)

0.923828125

## Currency Rates

In [122]:
dtt.now()

datetime.datetime(2019, 5, 12, 3, 24, 43, 831860)

In [123]:
date_obj = dtt.now()
c = CurrencyRates()

In [27]:
c.get_rates('USD',date_obj)

{'BGN': 1.7415850401,
 'NZD': 1.5160284951,
 'ILS': 3.5622439893,
 'RUB': 65.3154942119,
 'CAD': 1.3474621549,
 'USD': 1.0,
 'PHP': 52.2902938557,
 'CHF': 1.0131789849,
 'AUD': 1.4300089047,
 'JPY': 109.750667854,
 'TRY': 6.1297417631,
 'HKD': 7.8481745325,
 'MYR': 4.1657168299,
 'HRK': 6.5975066785,
 'CZK': 22.9136242208,
 'IDR': 14405.5031166518,
 'DKK': 6.6480854853,
 'NOK': 8.7438112199,
 'HUF': 288.0854853072,
 'GBP': 0.768032057,
 'MXN': 19.1975957257,
 'THB': 31.5796972395,
 'ISK': 121.9946571683,
 'ZAR': 14.2263579697,
 'BRL': 3.9541406946,
 'SGD': 1.3628673197,
 'PLN': 3.8254674978,
 'INR': 69.9977738201,
 'KRW': 1180.160284951,
 'RON': 4.2384683882,
 'CNY': 6.8235084595,
 'SEK': 9.6267141585,
 'EUR': 0.8904719501}

In [2]:
% ls

 Volume in drive C is OS
 Volume Serial Number is 6CCF-598B

 Directory of C:\Users\work-02\Documents\git\hackdelft2019

11-May-19  04:37 PM    <DIR>          .
11-May-19  04:37 PM    <DIR>          ..
11-May-19  04:31 PM    <DIR>          .ipynb_checkpoints
11-May-19  04:33 PM        20,646,912 Hackatondata.xls
11-May-19  04:37 PM             1,300 Untitled.ipynb
               2 File(s)     20,648,212 bytes
               3 Dir(s)  14,602,547,200 bytes free
